In [1]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, LongType, IntegerType, BooleanType

types = {
    'Float': lambda: FloatType(),
    'Integer': lambda: LongType(),
    'Long': lambda: LongType(),
    'Bool': lambda: IntegerType()
}

feats = []
with open('../CSV.header.txt') as header_file:
    for lineno, line in enumerate(header_file):
        line = line.strip()
        if ':' in line:
            feat_name, type_name = line.split(':')
            feats.append(StructField(feat_name, types[type_name](), True))
        else:
            feats.append(StructField(line, StringType(), True))
            
schema = StructType(feats)
feats

[StructField(GLOBALEVENTID,LongType,true),
 StructField(SQLDATE,LongType,true),
 StructField(MonthYear,LongType,true),
 StructField(Year,LongType,true),
 StructField(FractionDate,FloatType,true),
 StructField(Actor1Code,StringType,true),
 StructField(Actor1Name,StringType,true),
 StructField(Actor1CountryCode,StringType,true),
 StructField(Actor1KnownGroupCode,StringType,true),
 StructField(Actor1EthnicCode,StringType,true),
 StructField(Actor1Religion1Code,StringType,true),
 StructField(Actor1Religion2Code,StringType,true),
 StructField(Actor1Type1Code,StringType,true),
 StructField(Actor1Type2Code,StringType,true),
 StructField(Actor1Type3Code,StringType,true),
 StructField(Actor2Code,StringType,true),
 StructField(Actor2Name,StringType,true),
 StructField(Actor2CountryCode,StringType,true),
 StructField(Actor2KnownGroupCode,StringType,true),
 StructField(Actor2EthnicCode,StringType,true),
 StructField(Actor2Religion1Code,StringType,true),
 StructField(Actor2Religion2Code,StringType,

In [2]:
df = spark.read.csv('/home/ubuntu/data-p3-d2/2015-unzip/2015*.gz', sep = '\t', schema=schema)

In [3]:
df.createOrReplaceTempView("GDELT")

<h2>Happiness and Sadness of Thai population In 2015</h2>
<p>The attributes that are specific to this analysis are 'AvgTone' representing 'emotion/sentiment', 'EventRootCode' representing 'type of events' and ActionGeo_CountryCode="TH" representing 'Thailand'</p>
<body>
    <center><h4>Sadness</h4></center>
<table>
<tr>
    <th>
        EventRootCode
    </th>
    <th>
        Event
    </th>
    <th>
        AvgTone
    </th>
</tr>
<tr>
    <td>
        18
    </td>
    <td>
        ASSAULT
    </td>
    <td> 
       -5.390966580175084 
    </td>
    </tr>
    <tr>
    <td>
        17
    </td>
    <td>
        COERCE
    </td>
    <td> 
       -4.980801102637645 
    </td>
    </tr>
       <tr>
    <td>
        19
    </td>
    <td>
        FIGHT
    </td>
    <td> 
       -4.61264282794244
    </td>
    </tr>
         <tr>
    <td>
        14
    </td>
    <td>
        PROTEST
    </td>
    <td> 
       -4.1006759595536755
    </td>
    </tr>
          <tr>
    <td>
        20
    </td>
    <td>
        ENGAGE IN UNCONVENTIONAL MASS VIOLENCE
    </td>
    <td> 
        -4.062580829729205
    </td>
    </tr>
</table>
        <center><h4>Happiness</h4></center>
<table>
<tr>
    <th>
        EventRootCode
    </th>
    <th>
        Event
    </th>
    <th>
        AvgTone
    </th>
</tr>
<tr>
    <td>
        5
    </td>
    <td>
        ENGAGE IN DIPLOMATIC COOPERATION
    </td>
    <td> 
       0.2415000501685426 
    </td>
    </tr>
    <tr>
    <td>
        3
    </td>
    <td>
        EXPRESS INTENT TO COOPERATE
    </td>
    <td> 
       -0.20847577040658494 
    </td>
    </tr>
       <tr>
    <td>
        4
    </td>
    <td>
        CONSULT
    </td>
    <td> 
       -1.024762808470614
    </td>
    </tr>
         <tr>
    <td>
        6
    </td>
    <td>
         ENGAGE IN MATERIAL COOPERATION
    </td>
    <td> 
       -1.350121166321724
    </td>
    </tr>
          <tr>
    <td>
        7
    </td>
    <td>
        PROVIDE AID
    </td>
    <td> 
        -1.4913063709509484
    </td>
    </tr>
</table>
</body>
<p>As you can see, from average tone even events which make people happy still got negative value so, I decided to go deeper to find which event makes people feel happiest and which one makes people feel saddest.</p>

In [4]:
th_avgtone=spark.sql('SELECT AVG(AvgTone) as Avg_Avgtone,EventRootCode from GDELT WHERE ActionGeo_CountryCode="TH" group by EventRootCode order by Avg_Avgtone Limit 5').show()
th_avgtone

+-------------------+-------------+
|        Avg_Avgtone|EventRootCode|
+-------------------+-------------+
| -5.390966580175084|           18|
| -4.980801102637645|           17|
|  -4.61264282794244|           19|
|-4.1006759595536755|           14|
| -4.062580829729205|           20|
+-------------------+-------------+



In [5]:
th_avgtone=spark.sql('SELECT AVG(AvgTone) as Avg_Avgtone,EventRootCode from GDELT WHERE ActionGeo_CountryCode="TH" group by EventRootCode order by Avg_Avgtone DESC Limit 5').show()
th_avgtone

+--------------------+-------------+
|         Avg_Avgtone|EventRootCode|
+--------------------+-------------+
|  0.2415000501685426|           05|
|-0.20847577040658494|           03|
|  -1.024762808470614|           04|
|  -1.350121166321724|           06|
| -1.4913063709509484|           07|
+--------------------+-------------+



<h2>The event which makes people happy the most is INVESTIGATE which AvgTone is  22.077923.</h2>

In [9]:
max_avgtone=spark.sql('SELECT max(AvgTone),EventRootCode from GDELT WHERE ActionGeo_CountryCode="TH" group by EventRootCode').show()
max_avgtone

+------------+-------------+
|max(AvgTone)|EventRootCode|
+------------+-------------+
|   12.314225|           07|
|   7.0422535|           15|
|   15.957447|           11|
|   13.768116|           01|
|   10.638298|           16|
|   10.869565|           18|
|   12.396694|           17|
|   22.077923|           09|
|   18.023256|           05|
|   10.185185|           19|
|   13.380281|           08|
|   13.577586|           03|
|    11.61413|           02|
|   12.765958|           06|
|   3.6281178|           20|
|   11.396104|           10|
|    9.178744|           12|
|   17.171717|           04|
|    9.219858|           13|
|        10.0|           14|
+------------+-------------+



<h2>The event which makes people sad the most are COERCE and FIGHT which have equally AvgTone , -23.913044.</h2>

In [10]:
min_avgtone=spark.sql('SELECT min(AvgTone),EventRootCode from GDELT WHERE ActionGeo_CountryCode="TH" group by EventRootCode').show()
min_avgtone

+------------+-------------+
|min(AvgTone)|EventRootCode|
+------------+-------------+
|  -16.030535|           07|
|       -12.5|           15|
|  -19.858156|           11|
|       -17.5|           01|
|   -18.27411|           16|
|   -20.86331|           18|
|  -23.913044|           17|
|   -17.79398|           09|
|  -16.464472|           05|
|  -23.913044|           19|
|  -16.777428|           08|
|  -16.304348|           03|
|   -18.77551|           02|
|  -15.126812|           06|
|  -11.490684|           20|
|   -16.04278|           10|
|  -19.155207|           12|
|   -17.80822|           04|
|  -16.296297|           13|
|  -16.083916|           14|
+------------+-------------+



In [28]:
test=spark.sql('select AvgTone,SOURCEURL from GDELT where ActionGeo_CountryCode="TH" and EventRootCode=09 order by AvgTone DESC').show(5,False)
test

+---------+-----------------------------------------------------------------------------------------------------------+
|AvgTone  |SOURCEURL                                                                                                  |
+---------+-----------------------------------------------------------------------------------------------------------+
|22.077923|http://sgblogs.com/pages/63409732-star-alliance-refreshes-website                                          |
|14.285714|http://sgblogs.com/pages/65162876-thailands-prince-of-dance-hunz-models-for-suit-select-in                 |
|11.494253|http://aida-batrisyia.blogspot.com/2015/12/zero-tolerance-2015.html                                        |
|10.891089|http://blog.traveluniversally.com/2015/02/top-4-tourist-destinations-in-thailand.html                      |
|10.588235|http://www.briefingwire.com/pr/thailand-holiday-homes-guides-travelers-in-finding-luxury-villas-in-kohsamui|
+---------+-----------------------------

In [ ]:
# from pyspark import SparkContext
# from pyspark.sql import Row
# rdd = sc.parallelize(new_list).map(lambda x: Row(feature1=x[0], feature2=x[1], pcc=x[2]))
# df_pcc = sqlContext.createDataFrame(rdd)
# df_pcc.createOrReplaceTempView("PCC")

In [ ]:
test_1=spark.sql('select * from GDELT where ActionGeo_CountryCode="TH" and EventRootCode=09 and AvgTone = 22.077923').show()
test_1

In [ ]:
test_2=spark.sql('select AvgTone,SOURCEURL from GDELT where ActionGeo_CountryCode="TH" and EventRootCode=09 order by AvgTone ASC').show(5,False)
test_2